Face Aging Using Conditional GAN

Download the dataset from the following link https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/

Import the modules.

In [1]:
import os
import os
import time
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras import Input, Model
from keras.applications import InceptionResNetV2
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, Flatten, Dense, BatchNormalization, Reshape, concatenate, LeakyReLU, Lambda, \
    K, Activation, UpSampling2D, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras_preprocessing import image
from scipy.io import loadmat

Using TensorFlow backend.


The encoder network is a convolutional neural network (CNN) that encodes an image (x) to a latent vector (z) or a latent vector representation.

In [2]:
def build_encoder():
    """
    Encoder Network
    """
    input_layer = Input(shape=(64, 64, 3))

    # 1st Convolutional Block
    enc = Conv2D(filters=32, kernel_size=5, strides=2, padding='same')(input_layer)
    # enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # 2nd Convolutional Block
    enc = Conv2D(filters=64, kernel_size=5, strides=2, padding='same')(enc)
    enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # 3rd Convolutional Block
    enc = Conv2D(filters=128, kernel_size=5, strides=2, padding='same')(enc)
    enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # 4th Convolutional Block
    enc = Conv2D(filters=256, kernel_size=5, strides=2, padding='same')(enc)
    enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # Flatten layer
    enc = Flatten()(enc)

    # 1st Fully Connected Layer
    enc = Dense(4096)(enc)
    enc = BatchNormalization()(enc)
    enc = LeakyReLU(alpha=0.2)(enc)

    # Second Fully Connected Layer
    enc = Dense(100)(enc)

    # Create a model
    model = Model(inputs=[input_layer], outputs=[enc])
    return model

The generator network is a CNN that takes a 100-dimensional vector z and generates an image with a dimension of (64, 64, 3). Let's implement the generator network in the Keras framework.

In [3]:
def build_generator():
    """
    Create a Generator Model with hyperparameters values defined as follows
    """
    latent_dims = 100
    num_classes = 6

    input_z_noise = Input(shape=(latent_dims,))
    input_label = Input(shape=(num_classes,))

    x = concatenate([input_z_noise, input_label])

    x = Dense(2048, input_dim=latent_dims + num_classes)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)

    x = Dense(256 * 8 * 8)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)

    x = Reshape((8, 8, 256))(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(filters=128, kernel_size=5, padding='same')(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(filters=64, kernel_size=5, padding='same')(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(filters=3, kernel_size=5, padding='same')(x)
    x = Activation('tanh')(x)

    model = Model(inputs=[input_z_noise, input_label], outputs=[x])
    return model

Expand label_input so that it has a shape of (32, 32, 6)
* The preceding function will transform a tensor with a dimension of (6, ) to a tensor with a dimension of (32, 32, 6)

In [4]:
def expand_label_input(x):
    x = K.expand_dims(x, axis=1)
    x = K.expand_dims(x, axis=1)
    x = K.tile(x, [1, 32, 32, 1])
    return x

The discriminator network is a CNN.

In [ ]:
def build_discriminator():
    """
    Create a Discriminator Model with hyperparameters values defined as follows
    """
    input_shape = (64, 64, 3)
    label_shape = (6,)
    image_input = Input(shape=input_shape)
    label_input = Input(shape=label_shape)

    x = Conv2D(64, kernel_size=3, strides=2, padding='same')(image_input)
    x = LeakyReLU(alpha=0.2)(x)

    label_input1 = Lambda(expand_label_input)(label_input)
    x = concatenate([x, label_input1], axis=3)

    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(512, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[image_input, label_input], outputs=[x])
    return model

Build the face recognition combined network.

In [ ]:
def build_fr_combined_network(encoder, generator, fr_model):
    input_image = Input(shape=(64, 64, 3))
    input_label = Input(shape=(6,))

    latent0 = encoder(input_image)

    gen_images = generator([latent0, input_label])

    fr_model.trainable = False

    resized_images = Lambda(lambda x: K.resize_images(gen_images, height_factor=2, width_factor=2,
                                                      data_format='channels_last'))(gen_images)
    embeddings = fr_model(resized_images)

    model = Model(inputs=[input_image, input_label], outputs=[embeddings])
    return model

Build the face recognition model.

In [ ]:
def build_fr_model(input_shape):
    resent_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape, pooling='avg')
    image_input = resent_model.input
    x = resent_model.layers[-1].output
    out = Dense(128)(x)
    embedder_model = Model(inputs=[image_input], outputs=[out])

    input_layer = Input(shape=input_shape)

    x = embedder_model(input_layer)
    output = Lambda(lambda x: K.l2_normalize(x, axis=-1))(x)

    model = Model(inputs=[input_layer], outputs=[output])
    return model

Utility functions to resize image and to calculate the age of the person from the serial date number and the year the photo was taken.

In [ ]:
#resize the images from a shape of (64, 64, 3) to a shape of (192, 192, 3)
def build_image_resizer():
    input_layer = Input(shape=(64, 64, 3))

    resized_images = Lambda(lambda x: K.resize_images(x, height_factor=3, width_factor=3,
                                                      data_format='channels_last'))(input_layer)

    model = Model(inputs=[input_layer], outputs=[resized_images])
    return model


def calculate_age(taken, dob):
    birth = datetime.fromordinal(max(int(dob) - 366, 1))

    if birth.month < 7:
        return taken - birth.year
    else:
        return taken - birth.year - 1

load_data method to load .mat files in Python. The photo_taken variable is a list of years and dob is Matlab's serial date number for the corresponding photo in the list. We can calculate the age of the person from the serial date number and the year the photo was taken using calculate_age method.

In [ ]:
def load_data(wiki_dir, dataset='wiki'):
    # Load the wiki.mat file
    meta = loadmat(os.path.join(wiki_dir, "{}.mat".format(dataset)))

    # Load the list of all files
    full_path = meta[dataset][0, 0]["full_path"][0]

    # List of Matlab serial date numbers
    dob = meta[dataset][0, 0]["dob"][0]

    # List of years when photo was taken
    photo_taken = meta[dataset][0, 0]["photo_taken"][0]  # year

    # Calculate age for all dobs
    age = [calculate_age(photo_taken[i], dob[i]) for i in range(len(dob))]

    # Create a list of tuples containing a pair of an image path and age
    images = []
    age_list = []
    for index, image_path in enumerate(full_path):
        images.append(image_path[0])
        age_list.append(age[index])

    # Return a list of all images and respective age
    return images, age_list


To convert the age numerical value to the age category.

In [ ]:
def age_to_category(age_list):
    print("age_to_category age_list size : " + str(len(age_list[:1000])))
    age_list1 = []

    for age in age_list:
        if 0 < age <= 18:
            age_category = 0
        elif 18 < age <= 29:
            age_category = 1
        elif 29 < age <= 39:
            age_category = 2
        elif 39 < age <= 49:
            age_category = 3
        elif 49 < age <= 59:
            age_category = 4
        elif age >= 60:
            age_category = 5

        age_list1.append(age_category)

    return age_list1

To load all images and create an ndarray containing all images.

In [ ]:
def load_images(data_dir, image_paths, image_shape):
    images = None
    for i, image_path in enumerate(image_paths):
        print("index i : " + str(i) + " and image path : " + str(image_path))
        try:
            # Load image
            loaded_image = image.load_img(os.path.join(data_dir, image_path), target_size=image_shape)

            # Convert PIL image to numpy ndarray
            loaded_image = image.img_to_array(loaded_image)

            # Add another dimension (Add batch dimension)
            loaded_image = np.expand_dims(loaded_image, axis=0)

            # Concatenate all images into one tensor
            if images is None:
                images = loaded_image
            else:
                images = np.concatenate([images, loaded_image], axis=0)
        except Exception as e:
            print("Error:", i, e)

    return images

To calculate Euclidean Distance Loss and other utility function.

In [ ]:
def euclidean_distance_loss(y_true, y_pred):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))


def write_log(callback, name, value, batch_no):
    summary = tf.Summary()
    summary_value = summary.value.add()
    summary_value.simple_value = value
    summary_value.tag = name
    callback.writer.add_summary(summary, batch_no)
    callback.writer.flush()

def save_rgb_img(img, path):
    """
    Save an rgb image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")

    plt.savefig(path)
    plt.close()

Let's start training. Here I run for the 50 EPOCHS and I stop running after 40 EPOCHS after, for the better result the value of EPOCHS need to be more like around 500.
Total number of images : 62328, I use onely 1000 images.

In [ ]:
if __name__ == '__main__':
    # Define hyperparameters
    data_dir = "./data"
    wiki_dir = os.path.join(data_dir, "wiki_crop")
    #epochs = 500
    epochs = 50
    batch_size = 2
    image_shape = (64, 64, 3)
    z_shape = 100
    TRAIN_GAN = True
    TRAIN_ENCODER = False
    TRAIN_GAN_WITH_FR = False
    fr_image_shape = (192, 192, 3)

    # Define optimizers
    # Optimizer for the discriminator network
    dis_optimizer = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999, epsilon=10e-8)
    # Optimizer for the generator network
    gen_optimizer = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999, epsilon=10e-8)
    # Optimizer for the adversarial network
    adversarial_optimizer = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999, epsilon=10e-8)

    """
    Build and compile networks
    """
    # Build and compile the discriminator network
    discriminator = build_discriminator()
    discriminator.compile(loss=['binary_crossentropy'], optimizer=dis_optimizer)

    # Build and compile the generator network
    generator = build_generator()
    generator.compile(loss=['binary_crossentropy'], optimizer=gen_optimizer)

    # Build and compile the adversarial model
    discriminator.trainable = False
    input_z_noise = Input(shape=(100,))
    input_label = Input(shape=(6,))
    recons_images = generator([input_z_noise, input_label])
    valid = discriminator([recons_images, input_label])
    adversarial_model = Model(inputs=[input_z_noise, input_label], outputs=[valid])
    adversarial_model.compile(loss=['binary_crossentropy'], optimizer=gen_optimizer)

    tensorboard = TensorBoard(log_dir="logs/{}".format(time.time()))
    tensorboard.set_model(generator)
    tensorboard.set_model(discriminator)

    """
    Load the dataset
    """
    images, age_list = load_data(wiki_dir=wiki_dir, dataset="wiki")
    print("number of images : "+ str(len(images)))
    print("age_list size : " + str(len(age_list[:1000])))
    age_cat = age_to_category(age_list)
    final_age_cat = np.reshape(np.array(age_cat), [len(age_cat), 1])
    classes = len(set(age_cat))
    y = to_categorical(final_age_cat, num_classes=len(set(age_cat)))

    loaded_images = load_images(wiki_dir, images[:1000], (image_shape[0], image_shape[1]))

    # Implement label smoothing
    real_labels = np.ones((batch_size, 1), dtype=np.float32) * 0.9
    fake_labels = np.zeros((batch_size, 1), dtype=np.float32) * 0.1

    """
    Train the generator and the discriminator network
    """
    if TRAIN_GAN:
        for epoch in range(epochs):
            print("Epoch:{}".format(epoch))

            gen_losses = []
            dis_losses = []

            number_of_batches = int(len(loaded_images) / batch_size)
            print("Number of batches:", number_of_batches)
            for index in range(number_of_batches):
                print("Batch:{}".format(index + 1))

                images_batch = loaded_images[index * batch_size:(index + 1) * batch_size]
                images_batch = images_batch / 127.5 - 1.0
                images_batch = images_batch.astype(np.float32)

                y_batch = y[index * batch_size:(index + 1) * batch_size]
                z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))

                """
                Train the discriminator network
                """

                # Generate fake images
                initial_recon_images = generator.predict_on_batch([z_noise, y_batch])

                d_loss_real = discriminator.train_on_batch([images_batch, y_batch], real_labels)
                d_loss_fake = discriminator.train_on_batch([initial_recon_images, y_batch], fake_labels)

                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                print("d_loss:{}".format(d_loss))

                """
                Train the generator network
                """

                z_noise2 = np.random.normal(0, 1, size=(batch_size, z_shape))
                random_labels = np.random.randint(0, 6, batch_size).reshape(-1, 1)
                random_labels = to_categorical(random_labels, 6)

                g_loss = adversarial_model.train_on_batch([z_noise2, random_labels], [1] * batch_size)

                print("g_loss:{}".format(g_loss))

                gen_losses.append(g_loss)
                dis_losses.append(d_loss)

            # Write losses to Tensorboard
            write_log(tensorboard, 'g_loss', np.mean(gen_losses), epoch)
            write_log(tensorboard, 'd_loss', np.mean(dis_losses), epoch)

            """
            Generate images after every 10th epoch
            """
            if epoch % 10 == 0:
                images_batch = loaded_images[0:batch_size]
                images_batch = images_batch / 127.5 - 1.0
                images_batch = images_batch.astype(np.float32)

                y_batch = y[0:batch_size]
                z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))

                gen_images = generator.predict_on_batch([z_noise, y_batch])

                for i, img in enumerate(gen_images[:5]):
                    save_rgb_img(img, path="./results/img_{}_{}.png".format(epoch, i))

        # Save networks
        try:
            generator.save_weights("generator.h5")
            discriminator.save_weights("discriminator.h5")
        except Exception as e:
            print("Error:", e)

    """
    Train encoder
    """

    if TRAIN_ENCODER:
        # Build and compile encoder
        encoder = build_encoder()
        encoder.compile(loss=euclidean_distance_loss, optimizer='adam')

        # Load the generator network's weights
        try:
            generator.load_weights("generator.h5")
        except Exception as e:
            print("Error:", e)

        z_i = np.random.normal(0, 1, size=(5000, z_shape))

        y = np.random.randint(low=0, high=6, size=(5000,), dtype=np.int64)
        num_classes = len(set(y))
        y = np.reshape(np.array(y), [len(y), 1])
        y = to_categorical(y, num_classes=num_classes)

        for epoch in range(epochs):
            print("Epoch:", epoch)

            encoder_losses = []

            number_of_batches = int(z_i.shape[0] / batch_size)
            print("Number of batches:", number_of_batches)
            for index in range(number_of_batches):
                print("Batch:", index + 1)

                z_batch = z_i[index * batch_size:(index + 1) * batch_size]
                y_batch = y[index * batch_size:(index + 1) * batch_size]

                generated_images = generator.predict_on_batch([z_batch, y_batch])

                # Train the encoder model
                encoder_loss = encoder.train_on_batch(generated_images, z_batch)
                print("Encoder loss:", encoder_loss)

                encoder_losses.append(encoder_loss)

            # Write the encoder loss to Tensorboard
            write_log(tensorboard, "encoder_loss", np.mean(encoder_losses), epoch)

        # Save the encoder model
        encoder.save_weights("encoder.h5")

    """
    Optimize the encoder and the generator network
    """
    if TRAIN_GAN_WITH_FR:

        # Load the encoder network
        encoder = build_encoder()
        encoder.load_weights("encoder.h5")

        # Load the generator network
        generator.load_weights("generator.h5")

        image_resizer = build_image_resizer()
        image_resizer.compile(loss=['binary_crossentropy'], optimizer='adam')

        # Face recognition model
        fr_model = build_fr_model(input_shape=fr_image_shape)
        fr_model.compile(loss=['binary_crossentropy'], optimizer="adam")

        # Make the face recognition network as non-trainable
        fr_model.trainable = False

        # Input layers
        input_image = Input(shape=(64, 64, 3))
        input_label = Input(shape=(6,))

        # Use the encoder and the generator network
        latent0 = encoder(input_image)
        gen_images = generator([latent0, input_label])

        # Resize images to the desired shape
        resized_images = Lambda(lambda x: K.resize_images(gen_images, height_factor=3, width_factor=3,
                                                          data_format='channels_last'))(gen_images)
        embeddings = fr_model(resized_images)

        # Create a Keras model and specify the inputs and outputs for the network
        fr_adversarial_model = Model(inputs=[input_image, input_label], outputs=[embeddings])

        # Compile the model
        fr_adversarial_model.compile(loss=euclidean_distance_loss, optimizer=adversarial_optimizer)

        for epoch in range(epochs):
            print("Epoch:", epoch)

            reconstruction_losses = []

            number_of_batches = int(len(loaded_images) / batch_size)
            print("Number of batches:", number_of_batches)
            for index in range(number_of_batches):
                print("Batch:", index + 1)

                images_batch = loaded_images[index * batch_size:(index + 1) * batch_size]
                images_batch = images_batch / 127.5 - 1.0
                images_batch = images_batch.astype(np.float32)

                y_batch = y[index * batch_size:(index + 1) * batch_size]

                images_batch_resized = image_resizer.predict_on_batch(images_batch)

                real_embeddings = fr_model.predict_on_batch(images_batch_resized)

                reconstruction_loss = fr_adversarial_model.train_on_batch([images_batch, y_batch], real_embeddings)

                print("Reconstruction loss:", reconstruction_loss)

                reconstruction_losses.append(reconstruction_loss)

            # Write the reconstruction loss to Tensorboard
            write_log(tensorboard, "reconstruction_loss", np.mean(reconstruction_losses), epoch)

            """
            Generate images
            """
            if epoch % 10 == 0:
                images_batch = loaded_images[0:batch_size]
                images_batch = images_batch / 127.5 - 1.0
                images_batch = images_batch.astype(np.float32)

                y_batch = y[0:batch_size]
                z_noise = np.random.normal(0, 1, size=(batch_size, z_shape))

                gen_images = generator.predict_on_batch([z_noise, y_batch])

                for i, img in enumerate(gen_images[:5]):
                    save_rgb_img(img, path="./results/img_opt_{}_{}.png".format(epoch, i))

        # Save improved weights for both of the networks
        generator.save_weights("generator_optimized.h5")
        encoder.save_weights("encoder_optimized.h5")

number of images : 62328
age_list size : 1000
age_to_category age_list size : 1000
index i : 0 and image path : 17/10000217_1981-05-05_2009.jpg
index i : 1 and image path : 48/10000548_1925-04-04_1964.jpg
index i : 2 and image path : 12/100012_1948-07-03_2008.jpg
index i : 3 and image path : 65/10001965_1930-05-23_1961.jpg
index i : 4 and image path : 16/10002116_1971-05-31_2012.jpg
index i : 5 and image path : 02/10002702_1960-11-09_2012.jpg
index i : 6 and image path : 41/10003541_1937-09-27_1971.jpg
index i : 7 and image path : 39/100039_1904-12-07_1982.jpg
index i : 8 and image path : 13/10004113_1946-08-26_2007.jpg
index i : 9 and image path : 22/10004122_1982-03-17_2011.jpg
index i : 10 and image path : 99/10004299_1908-08-19_1950.jpg
index i : 11 and image path : 56/1000456_1933-06-12_1969.jpg
index i : 12 and image path : 82/10004882_1987-05-16_2010.jpg
index i : 13 and image path : 22/1000522_1931-01-15_1959.jpg
index i : 14 and image path : 61/10005261_1936-06-24_1974.jpg
ind

index i : 137 and image path : 71/10071_1961-04-03_2010.jpg
index i : 138 and image path : 17/1007217_1937-05-26_2000.jpg
index i : 139 and image path : 98/10072598_1972-07-06_2009.jpg
index i : 140 and image path : 15/10074515_1968-03-03_2010.jpg
index i : 141 and image path : 99/1007499_1983-03-09_2013.jpg
index i : 142 and image path : 01/10075401_1944-11-05_1982.jpg
index i : 143 and image path : 15/10075415_1988-01-11_2010.jpg
index i : 144 and image path : 99/10075699_1960-11-06_2011.jpg
index i : 145 and image path : 79/10075979_1954-11-15_1977.jpg
index i : 146 and image path : 36/1007636_1968-12-17_2012.jpg
index i : 147 and image path : 09/1007709_1922-04-15_1967.jpg
index i : 148 and image path : 64/1007764_1947-12-21_2007.jpg
index i : 149 and image path : 67/1007767_1899-12-14_1970.jpg
index i : 150 and image path : 65/10077765_1954-03-07_2010.jpg
index i : 151 and image path : 20/10078120_1948-06-07_1940.jpg
index i : 152 and image path : 81/10078681_1940-10-01_1940.jpg
i

index i : 270 and image path : 87/20050887_1973-07-21_1996.jpg
index i : 271 and image path : 67/20050967_1974-01-08_1995.jpg
index i : 272 and image path : 15/20051115_1981-04-27_2011.jpg
index i : 273 and image path : 90/2005290_1972-10-22_2008.jpg
index i : 274 and image path : 72/20053572_1982-04-06_2009.jpg
index i : 275 and image path : 58/20054558_1935-08-25_1956.jpg
index i : 276 and image path : 74/2005574_1956-11-21_2009.jpg
index i : 277 and image path : 63/200563_1971-05-27_2014.jpg
index i : 278 and image path : 91/200591_1947-01-24_1978.jpg
index i : 279 and image path : 86/20060786_1919-01-08_1942.jpg
index i : 280 and image path : 04/20061004_1985-04-05_2008.jpg
index i : 281 and image path : 62/20063362_1987-10-05_2011.jpg
index i : 282 and image path : 84/20063484_1987-08-13_2010.jpg
index i : 283 and image path : 61/2006361_1907-12-25_1945.jpg
index i : 284 and image path : 82/20064382_1986-10-12_2014.jpg
index i : 285 and image path : 75/2006475_1931-01-20_2007.jpg


index i : 403 and image path : 52/30050352_1959-03-14_2006.jpg
index i : 404 and image path : 72/3005072_1968-08-09_2009.jpg
index i : 405 and image path : 04/30052704_1986-10-22_2010.jpg
index i : 406 and image path : 61/300561_1949-08-20_1980.jpg
index i : 407 and image path : 34/30056734_1958-02-01_2007.jpg
index i : 408 and image path : 97/30057197_1992-02-29_2010.jpg
index i : 409 and image path : 13/30058713_1991-12-09_2014.jpg
index i : 410 and image path : 15/3005915_1980-08-25_2012.jpg
index i : 411 and image path : 34/30060834_1935-08-20_2005.jpg
index i : 412 and image path : 62/30061462_1991-03-12_2015.jpg
index i : 413 and image path : 79/30062079_1991-03-13_2010.jpg
index i : 414 and image path : 60/30062160_1968-11-26_2009.jpg
index i : 415 and image path : 55/30063955_1922-12-16_1948.jpg
index i : 416 and image path : 87/30064987_1992-11-19_2011.jpg
index i : 417 and image path : 50/300650_1894-09-15_1977.jpg
index i : 418 and image path : 13/30065213_1992-03-02_2012.jp

index i : 539 and image path : 50/4009050_1963-10-11_2009.jpg
index i : 540 and image path : 75/40091375_1988-01-15_2014.jpg
index i : 541 and image path : 32/40092132_1981-02-10_2014.jpg
index i : 542 and image path : 63/40092263_1988-01-29_2014.jpg
index i : 543 and image path : 92/40093192_1996-08-02_2014.jpg
index i : 544 and image path : 08/40093408_1989-09-27_2013.jpg
index i : 545 and image path : 59/40098159_1902-01-15_1988.jpg
index i : 546 and image path : 72/4009972_1967-01-08_2007.jpg
index i : 547 and image path : 82/4009982_1983-01-11_2011.jpg
index i : 548 and image path : 13/5001813_1917-07-01_1944.jpg
index i : 549 and image path : 62/5001962_1986-01-26_2009.jpg
index i : 550 and image path : 78/5002778_1959-02-18_2014.jpg
index i : 551 and image path : 73/5003173_1983-06-06_2013.jpg
index i : 552 and image path : 18/5004918_1982-12-01_2014.jpg
index i : 553 and image path : 16/5005816_1951-12-02_2015.jpg
index i : 554 and image path : 92/500592_1912-04-09_1980.jpg
ind

...

d_loss:0.20118765532970428
g_loss:5.343667984008789
Batch:488
d_loss:0.16764496266841888
g_loss:4.476121425628662
Batch:489
d_loss:0.17449790239334106
g_loss:5.5391950607299805
Batch:490
d_loss:0.35037991404533386
g_loss:3.512972831726074
Batch:491
d_loss:0.1778062880039215
g_loss:2.2096638679504395
Batch:492
d_loss:0.23250797390937805
g_loss:4.456676006317139
Batch:493
d_loss:0.16705802083015442
g_loss:2.1824498176574707
Batch:494
d_loss:0.17003518342971802
g_loss:2.4108757972717285
Batch:495
d_loss:0.2728678286075592
g_loss:2.322258234024048
Batch:496
d_loss:0.2193818837404251
g_loss:4.2698822021484375
Batch:497
d_loss:0.28039953112602234
g_loss:1.8202383518218994
Batch:498
d_loss:0.2697012424468994
g_loss:1.0787341594696045
Batch:499
d_loss:0.342367947101593
g_loss:4.287220478057861
Batch:500
d_loss:0.25400060415267944
g_loss:4.145604133605957
Epoch:39
Number of batches: 500
Batch:1
d_loss:0.3058331310749054
g_loss:2.3369195461273193
Batch:2
d_loss:0.23189759254455566
g_loss:2.28862

g_loss:4.028487205505371
Batch:121
d_loss:0.22136391699314117
g_loss:3.5651121139526367
Batch:122
d_loss:0.20545312762260437
g_loss:2.596132278442383
Batch:123
d_loss:0.2288302779197693
g_loss:2.2292377948760986
Batch:124
d_loss:0.4359516501426697
g_loss:3.072564125061035
Batch:125
d_loss:0.21165186166763306
g_loss:3.8412084579467773
Batch:126
d_loss:0.2311924397945404
g_loss:1.8458755016326904
Batch:127
d_loss:0.17870593070983887
g_loss:4.602608680725098
Batch:128
d_loss:0.17416808009147644
g_loss:3.055976390838623
Batch:129
d_loss:0.24577808380126953
g_loss:2.384305953979492
Batch:130
d_loss:0.2821084260940552
g_loss:3.2314257621765137
Batch:131
d_loss:0.2032511681318283
g_loss:2.4908151626586914
Batch:132
d_loss:0.3511956036090851
g_loss:3.8637187480926514
Batch:133
d_loss:0.18586984276771545
g_loss:3.169166088104248
Batch:134
d_loss:0.18774619698524475
g_loss:6.197317600250244
Batch:135
d_loss:0.16980449855327606
g_loss:5.481251239776611
Batch:136
d_loss:0.16463631391525269
g_loss:

d_loss:0.17980360984802246
g_loss:1.244516372680664
Batch:254
d_loss:0.21320295333862305
g_loss:4.287890911102295
Batch:255
d_loss:0.2251853495836258
g_loss:1.1829731464385986
Batch:256
d_loss:0.3461245000362396
g_loss:2.6453912258148193
Batch:257
d_loss:0.19825759530067444
g_loss:5.606113433837891
Batch:258
d_loss:0.23181742429733276
g_loss:5.747224807739258
Batch:259
d_loss:0.29893267154693604
g_loss:2.3041696548461914
Batch:260
d_loss:0.19326943159103394
g_loss:2.3249034881591797
Batch:261
d_loss:0.16925734281539917
g_loss:2.3860204219818115
Batch:262
d_loss:0.27725541591644287
g_loss:5.552038669586182
Batch:263
d_loss:0.2706383168697357
g_loss:2.854653835296631
Batch:264
d_loss:0.18064682185649872
g_loss:2.0478692054748535
Batch:265
d_loss:0.21418559551239014
g_loss:1.2839133739471436
Batch:266
d_loss:0.809296727180481
g_loss:3.4443962574005127
Batch:267
d_loss:0.16537770628929138
g_loss:7.09868860244751
Batch:268
d_loss:1.2748773097991943
g_loss:1.9301419258117676
Batch:269
d_loss

g_loss:2.912712574005127
Batch:386
d_loss:1.2342592477798462
g_loss:3.3569726943969727
Batch:387
d_loss:0.19515487551689148
g_loss:3.9427688121795654
Batch:388
d_loss:1.2231762409210205
g_loss:3.0348167419433594
Batch:389
d_loss:0.16735832393169403
g_loss:4.496337890625
Batch:390
d_loss:0.24109190702438354
g_loss:5.135229587554932
Batch:391
d_loss:0.36373570561408997
g_loss:2.713127851486206
Batch:392
d_loss:0.2916189432144165
g_loss:1.5066982507705688
Batch:393
d_loss:0.3287920355796814
g_loss:4.204744815826416
Batch:394
d_loss:0.20366977155208588
g_loss:3.5962483882904053
Batch:395
d_loss:0.21501222252845764
g_loss:4.145401954650879
Batch:396
d_loss:0.19553729891777039
g_loss:5.1749267578125
Batch:397
d_loss:0.25546786189079285
g_loss:2.710144519805908
Batch:398
d_loss:0.18729878962039948
g_loss:3.023324966430664
Batch:399
d_loss:0.22008250653743744
g_loss:2.1064541339874268
Batch:400
d_loss:0.2001197636127472
g_loss:4.528420925140381
Batch:401
d_loss:0.18458083271980286
g_loss:5.280

d_loss:0.24435317516326904
g_loss:5.676814079284668
Batch:19
d_loss:0.17183177173137665
g_loss:1.4933162927627563
Batch:20
d_loss:0.24593830108642578
g_loss:2.088731527328491
Batch:21
d_loss:0.16588427126407623
g_loss:4.069669723510742
Batch:22
d_loss:0.31458204984664917
g_loss:3.635864496231079
Batch:23
d_loss:0.260113000869751
g_loss:5.780872344970703
Batch:24
d_loss:0.4148341715335846
g_loss:2.1762161254882812
Batch:25
d_loss:0.2131040394306183
g_loss:4.582906723022461
Batch:26
d_loss:0.2130330204963684
g_loss:4.260457992553711
Batch:27
d_loss:0.2436809539794922
g_loss:2.984468936920166
Batch:28
d_loss:0.17836156487464905
g_loss:2.2436447143554688
Batch:29
d_loss:0.19900508224964142
g_loss:3.3252806663513184
Batch:30
d_loss:0.1691897213459015
g_loss:2.49404239654541
Batch:31
d_loss:0.19564110040664673
g_loss:1.062317132949829
Batch:32
d_loss:0.17517989873886108
g_loss:1.6806254386901855
Batch:33
d_loss:0.18014489114284515
g_loss:2.9102883338928223
Batch:34
d_loss:0.29115572571754456